In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

In [7]:
#read file: USD_CAD_M15_2005_202206
FILE_NAME_LIST = [
    'USD_CHF_M5_2005_202206','USD_CHF_M15_2005_202206','USD_CHF_M30_2005_202206',
    'AUD_CAD_M5_2005_202206','AUD_CAD_M15_2005_202206','AUD_CAD_M30_2005_202206',
    'AUD_JPY_M5_2005_202206','AUD_JPY_M15_2005_202206','AUD_JPY_M30_2005_202206',
    'AUD_USD_M5_2005_202206','AUD_USD_M15_2005_202206','AUD_USD_M30_2005_202206',
    'EUR_CAD_M5_2005_202206','EUR_CAD_M15_2005_202206','EUR_CAD_M30_2005_202206',
    'EUR_JPY_M5_2005_202206','EUR_JPY_M15_2005_202206','EUR_JPY_M30_2005_202206',
    'NZD_USD_M5_2005_202206','NZD_USD_M15_2005_202206','NZD_USD_M30_2005_202206',
    'USD_JPY_M5_2005_202206','USD_JPY_M15_2005_202206','USD_JPY_M30_2005_202206'
]

#change directroy
data_ls = []
for i in FILE_NAME_LIST:
    file = pd.read_csv(f'C:\\Users\\HFY\\Desktop\\capstone\\data\\Data_OANDA\\{i}.csv', encoding='utf-8')
    data_ls.append(file)

In [2]:
# len(data_ls)
data = pd.read_csv(f'C:\\Users\\HFY\\Desktop\\capstone\\data\\Data_OANDA\\AUD_JPY_M5_2005_202206.csv', encoding='utf-8')

data.reset_index(drop=True, inplace=True)

![rs.jpg](./rs.jpg)

In [5]:
#check for any missing value
file.isnull().sum()

datetime    0
open        0
high        0
low         0
close       0
dtype: int64

In [22]:

#use lastest 250 row of data for visual and pattern detect

data = data[0:800]

In [23]:
fig = go.Figure(data=[go.Candlestick(x=data['datetime'],
                open=data['open'],
                high=data['high'],
                low=data['low'],
                close=data['close'])])

fig.show()


In [3]:
def check_ascending_order(nums):
    count = 0
    if len(nums) != 1:
        for i in range(len(nums)):
            if nums[i] >= nums[0]:
                count+=1  
    if count == len(nums):
        return True
    else:
        return False

def check_descending_order(nums):
    count = 0
    if len(nums) != 1:
        for i in range(len(nums)):
            if nums[i] <= nums[0]:
                count+=1
    if count == len(nums):
        return True
    else:
        return False  


def converge(high_ls, low_ls):
    
    high_ls, low_ls = np.array(high_ls) , np.array(low_ls)
    # print('h_s' + str(high_ls))
    # print('l_s' + str(low_ls))
    x = [i for i in range(len(high_ls))]
    high_slope, high_intecept = np.polyfit(x,high_ls,1)
    low_slope, low_intecept = np.polyfit(x,low_ls,1)
    if high_slope < low_slope:
        return True
    else:
        return False

def converge2(high_ls, low_ls):
    
    high_ls, low_ls = np.array(high_ls) , np.array(low_ls)
    # print('h_s' + str(high_ls))
    # print('l_s' + str(low_ls))
    x = [i for i in range(len(high_ls))]
    high_slope, high_intecept = np.polyfit(x,high_ls,1)
    low_slope, low_intecept = np.polyfit(x,low_ls,1)
    if high_slope > low_slope:
        return True
    else:
        return False


def rising_wedge(df,high,low,open,close, datetime,length = 5):
    #length: minimum number of candlestick bar that satisfy the conditions
    high_ls = []
    rw_datetime_ls = []
    low_ls = []
    open_ls ,close_ls = [],[]
    try:
        df = df.reset_index()
    except:
        pass
    df['Rising Wedge'] = 0

    for i in range(len(high)):
    # rising wedge satisfy 3 conditions:
    # high and low both are rising
    # converge (trading volumne is decreasing)
        if check_ascending_order(high[i:i+length]) and check_ascending_order(low[i:i+length]) and converge(high[i:i+length],low[i:i+length]):
            high_ls.append(high[i:i+length])
            low_ls.append(low[i:i+length])
            rw_datetime_ls.append(datetime[i:i+length])
            open_ls.append(open[i:i+length])
            close_ls.append(close[i:i+length])
            # mark the dataframe falling wedge = 1, otherwise 0
            df['Rising Wedge'][i:i+length] = 1

    
    return df, high_ls, rw_datetime_ls, low_ls, open_ls,close_ls
        
def falling_wedge(df,high,low,open,close, datetime,length = 5):
    #length: minimum number of candlestick bar that satisfy the conditions
    low_ls = []
    fw_datetime_ls = []
    high_ls = []
    open_ls ,close_ls = [],[]
    try:
        df = df.reset_index()
    except:
        pass
    df['Falling Wedge'] = 0

    for i in range(len(high)):
    # falling wedge satisfy 3 conditions:
    # high and low both are falling
    # converge (trading volumne is decreasing)
        if  check_descending_order(high[i:i+length]) and check_descending_order(low[i:i+length]) and converge(high[i:i+length],low[i:i+length]):
            low_ls.append(low[i:i+length])
            high_ls.append(high[i:i+length])
            fw_datetime_ls.append(datetime[i:i+length])
            open_ls.append(open[i:i+length])
            close_ls.append(close[i:i+length])
            # mark the dataframe falling wedge = 1, otherwise 0
            df['Falling Wedge'][i:i+length] = 1

    return df, high_ls, fw_datetime_ls, low_ls, open_ls,close_ls

def analysis_rising_wedge(data):
# collect results into a list for visualization
    final_a, final_b, final_c ,final_d,final_e= [],[],[],[],[]
    for i in range(len(data['Rising Wedge'])-1):
        if data['Rising Wedge'][i] == 1:
            final_a.append(data['high'][i])
            final_b.append(data['datetime'][i])
            final_c.append(data['low'][i])
            final_d.append(data['open'][i])
            final_e.append(data['close'][i])
            if data['Rising Wedge'][i+1] == 0:
                # print('r')
                final_a.append('X')
                final_b.append('X')
                final_c.append('X')
                final_d.append('X')
                final_e.append('X')

    temp =[]
    result_high, result_low, result_datetime,result_open,result_close = [],[],[],[],[]

    for i in final_a:
        temp.append(i)
        if i =='X':
            temp.pop()
            result_high.append(temp)
            temp = []
    temp =[]
    for i in final_b:
        temp.append(i)
        if i =='X':
            temp.pop()
            result_datetime.append(temp)
            temp = []
    temp =[]
    for i in final_c:
        temp.append(i)
        if i =='X':
            temp.pop()
            result_low.append(temp)
            temp = []
    temp =[]
    for i in final_d:
        temp.append(i)
        if i =='X':
            temp.pop()
            result_open.append(temp)
            temp = []
    temp =[]
    for i in final_e:
        temp.append(i)
        if i =='X':
            temp.pop()
            result_close.append(temp)
            temp = []

    print(str(len(result_high)) + ' rising wedge patterns')

    return result_high, result_datetime, result_low,result_open,result_close

def analysis_falling_wedge(data):
# collect results into a list for visualization
    final_a, final_b, final_c ,final_d,final_e= [],[],[],[],[]
    for i in range(len(data['Falling Wedge'])-1):
        if data['Falling Wedge'][i] == 1:
            final_a.append(data['high'][i])
            final_b.append(data['datetime'][i])
            final_c.append(data['low'][i])
            final_d.append(data['open'][i])
            final_e.append(data['close'][i])
            if data['Falling Wedge'][i+1] == 0:
                # print('r')
                final_a.append('X')
                final_b.append('X')
                final_c.append('X')
                final_d.append('X')
                final_e.append('X')

    temp =[]
    result_high, result_low, result_datetime,result_open,result_close = [],[],[],[],[]

    for i in final_a:
        temp.append(i)
        if i =='X':
            temp.pop()
            result_high.append(temp)
            temp = []
    temp =[]
    for i in final_b:
        temp.append(i)
        if i =='X':
            temp.pop()
            result_datetime.append(temp)
            temp = []
    temp =[]
    for i in final_c:
        temp.append(i)
        if i =='X':
            temp.pop()
            result_low.append(temp)
            temp = []
    temp =[]
    for i in final_d:
        temp.append(i)
        if i =='X':
            temp.pop()
            result_open.append(temp)
            temp = []
    temp =[]
    for i in final_e:
        temp.append(i)
        if i =='X':
            temp.pop()
            result_close.append(temp)
            temp = []

    print(str(len(result_high)) + ' falling wedge patterns')

    return result_high, result_datetime, result_low,result_open,result_close

In [4]:
# Testing
data = data[0:500]
high = list(data['high'])
low = list(data['low'])
datetime =  list(data['datetime'])
open = list(data['open'])
close = list(data['close'])
data, high_ls, datetime_ls, low_ls,open_ls,close_ls = rising_wedge(data,high,low,open,close,datetime,length = 30)

result_high, result_datetime, result_low ,result_open,result_close= analysis_rising_wedge(data)
# print(result_low)

4 rising wedge patterns


In [7]:
def idx_max(nums):
    for i in range(len(nums)):
        if nums[i] == max(nums):
            return i

def idx_low(nums):
    for i in range(len(nums)):
        if nums[i] == min(nums):
            return i
            
for i in range(3):

    fig = go.Figure(data=[go.Candlestick(x=result_datetime[i],
                    open=result_open[i],
                    high=result_high[i],
                    low=result_low[i],
                    close=result_close[i])])

    slope_high, _ = np.polyfit([i+1 for i in range(len(result_high[i]))], result_high[i], 1)
    slope_low, _ = np.polyfit([i+1 for i in range(len(result_low[i]))], result_low[i], 1)

    print('low slope: ' + str(slope_low))
    print('high slope: ' + str(slope_high))

    adjust_intercept_high = max(result_high[i]) + 0.001* slope_high*idx_max(result_high[i])
    adjust_intercept_low  = min(result_low[i]) - slope_low*idx_low(result_low[i])

    y_high = [adjust_intercept_high] * len(result_high[i])
    #y_high = adjust_intercept_high + slope_high*np.array([i for i in range(len(result_high[i]))])
    y_low = adjust_intercept_low +  slope_low*np.array([i for i in range(len(result_low[i]))])

    fig.add_trace(go.Scatter(x=result_datetime[i], y= y_high , mode='lines', marker_color = 'green', showlegend=True, name = 'high slope'))
    fig.add_trace(go.Scatter(x=result_datetime[i], y= y_low, mode='lines', marker_color = 'blue',showlegend=True,name = 'low slope'))
    fig.update_layout(title="Rising Wedge")
    fig.show()
    # print( y_high)
    

low slope: 0.0034599429608509922
high slope: 0.002803344568317326


low slope: 0.009022551092319663
high slope: 0.007560958421424112


low slope: 0.004124839124838652
high slope: 0.004401544401543033


In [12]:
# idx = 0
# num_ls = []
# for data in data_ls:
#     high = list(data['high'])
#     low = list(data['low'])
#     datetime =  list(data['datetime'])
#     data, high_ls, datetime_ls, low_ls = rising_wedge(data,high,low,datetime,length = 5)
#     print(f'{FILE_NAME_LIST[idx]}: ', end = '')
#     result_high, result_datetime, result_low = analysis_rising_wedge(data)
#     num_ls.append(len(result_datetime))
#     idx += 1

# rising_wedge_table = pd.DataFrame(pd.Series(num_ls, index=FILE_NAME_LIST))

In [200]:
# Testing
data = data[0:800]
high = list(data['high'])
low = list(data['low'])
datetime =  list(data['datetime'])
open = list(data['open'])
close = list(data['close'])
data, result_high, result_datetime, result_low,result_open,result_close = falling_wedge(data,high,low,open,close,datetime,length = 30)

# result_high, result_datetime, result_low ,result_open,result_close= analysis_falling_wedge(data)

In [202]:
def idx_max(nums):
    for i in range(len(nums)):
        if nums[i] == max(nums):
            return i

def idx_low(nums):
    for i in range(len(nums)):
        if nums[i] == min(nums):
            return i
            
for i in range(3):

    fig = go.Figure(data=[go.Candlestick(x=result_datetime[i],
                    open=result_open[i],
                    high=result_high[i],
                    low=result_low[i],
                    close=result_close[i])])

    slope_high, _ = np.polyfit([i+1 for i in range(len(result_high[i]))], result_high[i], 1)
    slope_low, _ = np.polyfit([i+1 for i in range(len(result_low[i]))], result_low[i], 1)

    print('low slope: ' + str(slope_low))
    print('high slope: ' + str(slope_high))

    adjust_intercept_high = max(result_high[i]) -  slope_high*idx_max(result_high[i])
    adjust_intercept_low  = min(result_low[i]) - slope_low*idx_low(result_low[i])

 
    y_high = adjust_intercept_high + slope_high*np.array([i for i in range(len(result_high[i]))])
    y_low = adjust_intercept_low +  slope_low*np.array([i for i in range(len(result_low[i]))])

    fig.add_trace(go.Scatter(x=result_datetime[i], y= y_high , mode='lines', marker_color = 'green', showlegend=True, name = 'high slope'))
    fig.add_trace(go.Scatter(x=result_datetime[i], y= y_low, mode='lines', marker_color = 'blue',showlegend=True,name = 'low slope'))
    fig.update_layout(title="Falling Wedge")
    fig.show()

low slope: -0.002783092324804159
high slope: -0.0029632925472740435


low slope: -0.0024760845383745797
high slope: -0.002787541713013375


low slope: -0.010827586206895359
high slope: -0.01230255839821879


In [49]:
# ################# Visualization Purpose ####################

# high = list(data['high'])
# low = list(data['low'])
# datetime =  list(data['datetime'])
# data, high_ls, datetime_ls, low_ls = falling_wedge(data,high,low,datetime,length = 5)

# result_high, result_datetime, result_low = analysis_falling_wedge(data)

# fig = go.Figure(data=[go.Candlestick(x=data['datetime'],
#                 open=data['open'],
#                 high=data['high'],
#                 low=data['low'],
#                 close=data['close'])])


# fig.add_trace(go.Scatter(x=result_datetime[9], y=result_high[9] , mode='lines', marker_color = 'red', showlegend=False, name = 'Falling Wedge'))
# fig.add_trace(go.Scatter(x=result_datetime[9], y=result_low[9], mode='lines', marker_color = 'red',showlegend=False))

# fig.update_layout(title="Falling Wedge")
# fig.show()

In [177]:
# ################# Visualization Purpose ####################
# high = list(data['high'])
# low = list(data['low'])
# datetime =  list(data['datetime'])
# open = list(data['open'])
# close = list(data['close'])
# data, high_ls, datetime_ls, low_ls,open_ls,close_ls = rising_wedge(data,high,low,open,close,datetime,length = 3)

# result_high, result_datetime, result_low ,result_open,result_close= analysis_rising_wedge(data)
# i = 1
# # for i in range(len(datetime_ls)):
# fig = go.Figure(data=[go.Candlestick(x=datetime_ls[i],
#             open=open_ls[i],
#             high=high_ls[i],
#             low=low_ls[i],
#             close=close_ls[i])])

# # # for i in range(len(result_high)):
# # #     fig.add_trace(go.Scatter(x=result_datetime[i], y=result_high[i] , mode='lines', marker_color = 'green', showlegend=False, name = 'Rising Wedge'))
# # #     fig.add_trace(go.Scatter(x=result_datetime[i], y=result_low[i], mode='lines', marker_color = 'green',showlegend=False))

# fig.update_layout(title="Rising Wedge")
# fig.show()